In [5]:
import os
import tweepy

def credentials_dict() -> dict:
    return {
        "consumer_key": os.getenv("twitter_consumer_api_key"),
        "consumer_secret": os.getenv("twitter_consumer_api_secret"),
        "access_token": os.getenv("twitter_access_api_key"),
        "access_token_secret": os.getenv("twitter_access_api_secret")
    }
credentials = credentials_dict()

In [6]:
auth = tweepy.OAuthHandler(credentials["consumer_key"], credentials["consumer_secret"])
auth.set_access_token(credentials["access_token"], credentials["access_token_secret"])
api = tweepy.API(auth)
available_loc = api.available_trends()

Forbidden: 403 Forbidden
453 - You currently have Essential access which includes access to Twitter API v2 endpoints only. If you need access to this endpoint, you’ll need to apply for Elevated access via the Developer Portal. You can learn more here: https://developer.twitter.com/en/docs/twitter-api/getting-started/about-twitter-api#v2-access-leve

In [2]:
available_loc = api.available_trends()

Forbidden: 403 Forbidden
453 - You currently have Essential access which includes access to Twitter API v2 endpoints only. If you need access to this endpoint, you’ll need to apply for Elevated access via the Developer Portal. You can learn more here: https://developer.twitter.com/en/docs/twitter-api/getting-started/about-twitter-api#v2-access-leve

In [2]:
streaming_client = tweepy.StreamingClient(os.environ.get("bearer_token"))
bb = streaming_client.sample()

In [5]:
streaming_client.sample()
